In [4]:
from bitstring import BitArray, BitStream

In [22]:
input_file = open("samples/x264_test.264", "rb")
bitstream = BitArray(input_file)

start_code = '0x000001'


To find the starting position and the ending postion for each NALU interatively.

In [44]:
pos = 0
start_code_positions = []

while True:
    try:
        pos = bitstream.find(start_code, start=pos, bytealigned=True)[0]
        start_code_positions.append(pos)
        pos += 1
    except IndexError:
        break
start_code_positions.append(len(bitstream)) # add the last position
print(start_code_positions)
print(len(start_code_positions))


[8, 240, 312, 5824, 6680, 6864, 7024, 7184, 7344, 7568, 7744, 7904, 8064, 8288, 8464, 8624, 8776]
17


In [45]:
n = len(start_code_positions)
for i in range(n-1):
    print(f'[{i+1}] start={start_code_positions[i]} end={start_code_positions[i+1]}')
    print(bitstream[start_code_positions[i]:start_code_positions[i+1]])

[1] start=8 end=240
0x0000016764001eacd940a03db011000003000100000300328f162d9600
[2] start=240 end=312
0x00000168ebe3cb22c0
[3] start=312 end=5824
0x0000010605ffffaadc45e9bde6d948b7962cd820d923eeef78323634202d20636f7265203134382072323634332035633635373034202d20482e3236342f4d5045472d342041564320636f646563202d20436f70796c65667420323030332d32303135202d20687474703a2f2f7777772e766964656f6c616e2e6f7267...
[4] start=5824 end=6680
0x000001658884003bfffef74ebf029b54c22a0392570af6caa42659b9526b01f2a600000300000300000300908471ee6b95c93d668000002f6008d83261c814411710811c19e258428748ca0000030000030000030000030000030000030000030000030000030000978100
[5] start=6680 end=6864
0x000001419a246c437ffea7840000030000030000594000
[6] start=6864 end=7024
0x000001419e427885ff0000030000030000352100
[7] start=7024 end=7184
0x000001019e617442bf000003000003000047c000
[8] start=7184 end=7344
0x000001019e636a42bf000003000003000047c100
[9] start=7344 end=7568
0x000001419a6849a84168994c0867fffe9e1000000300000300015b00

Alternatively, the list of NAL units can be splited using BitArray.find()

In [39]:
nalus_ba = list(BitArray(input_file).split('0x000001', bytealigned=True))[1:]
# print(len(nalus_ba))
for nalu_ba in nalus_ba:
    print(nalu_ba)
    # enulation_prevention_three_byte
    nalu_ba.replace('0x000003', '0x0000', bytealigned=True)


0x0000016764001eacd940a03db011000003000100000300328f162d9600
0x00000168ebe3cb22c0
0x0000010605ffffaadc45e9bde6d948b7962cd820d923eeef78323634202d20636f7265203134382072323634332035633635373034202d20482e3236342f4d5045472d342041564320636f646563202d20436f70796c65667420323030332d32303135202d20687474703a2f2f7777772e766964656f6c616e2e6f7267...
0x000001658884003bfffef74ebf029b54c22a0392570af6caa42659b9526b01f2a600000300000300000300908471ee6b95c93d668000002f6008d83261c814411710811c19e258428748ca0000030000030000030000030000030000030000030000030000030000978100
0x000001419a246c437ffea7840000030000030000594000
0x000001419e427885ff0000030000030000352100
0x000001019e617442bf000003000003000047c000
0x000001019e636a42bf000003000003000047c100
0x000001419a6849a84168994c0867fffe9e1000000300000300015b00
0x000001419e8645112c2fff0000030000030000352100
0x000001019ea57442bf000003000003000047c100
0x000001019ea76a42bf000003000003000047c000
0x000001419aac49a8416c994c0857fffe384000000300000300054c00
0x000001419eca45